In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# input list for patient, we have 3 doctors, 5 patients in this case
patient1= [1,2,3,4]
patient2= [2,4,1,3]
patient3= [1,2,4,3]
patient4= [3,1,2,4]
patient5= [4,1,3,2]

#input: capacity for each doctor
capacity = 2
number_of_doctor=len(patient1)

matrix = np.row_stack((np.array(patient1).repeat(capacity), 
                       np.array(patient2).repeat(capacity),
                       np.array(patient3).repeat(capacity),
                       np.array(patient4).repeat(capacity),
                       np.array(patient5).repeat(capacity)))


# add dummy variables to make square matrix, with colnames = each spot for doctor, rownames = each spot for doctor
dum=0
while(matrix.shape[0] < matrix.shape[1]):
    dum+=1
    matrix=np.vstack([matrix,[0]*matrix.shape[1]])

print(matrix)
# KM algorithmn
row, col = linear_sum_assignment(matrix)

#best point in the matrix, minimized result
print(row,col)

start=0
for i in range(number_of_doctor):
    end= start+capacity
    l = list(range(start, end))
    start=end
    patient_num=(col[:, None] == l).argmax(axis=0)
    print("doctor"+ str(i)+" has patient "+ str(patient_num))

    
print(dum)

[[1 1 2 2 3 3 4 4]
 [2 2 4 4 1 1 3 3]
 [1 1 2 2 4 4 3 3]
 [3 3 1 1 2 2 4 4]
 [4 4 1 1 3 3 2 2]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[0 1 2 3 4 5 6 7] [0 4 1 2 3 5 6 7]
doctor0 has patient [0 2]
doctor1 has patient [3 4]
doctor2 has patient [1 5]
doctor3 has patient [6 7]
3


KM algorithmn

In [5]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# input list for patient, we have 3 doctors, 5 patients in this case
patient1= [1,2,3,4]
patient2= [2,4,1,3]
patient3= [1,2,4,3]
patient4= [3,1,2,4]
patient5= [4,1,3,2]

#input: capacity for each doctor
capacity = 2
number_of_doctor=len(patient1)

matrix = np.row_stack((np.array(patient1).repeat(capacity), 
                       np.array(patient2).repeat(capacity),
                       np.array(patient3).repeat(capacity),
                       np.array(patient4).repeat(capacity),
                       np.array(patient5).repeat(capacity)))


# add dummy variables to make square matrix, with colnames = each spot for doctor, rownames = each spot for doctor
dum=0
while(matrix.shape[0] < matrix.shape[1]):
    dum+=1
    matrix=np.vstack([matrix,[0]*matrix.shape[1]])
matrix=matrix.transpose()
#print(matrix)

# row reduction and column reduction
dim=matrix.shape[0]
cur_mat=matrix
for i in range(dim):
    cur_mat[i]=cur_mat[i]-np.min(cur_mat[i])
for i in range(dim):
    cur_mat[:,i]=cur_mat[:,i]-np.min(cur_mat[:,i])
#print(cur_mat)


def count_zero(cur_mat):
    num_zero_row=[]
    for i in range(cur_mat.shape[0]):
        row_zero = np.where(cur_mat[i].astype(int)==0)[0]
        num_zero_row.append(len(row_zero))

    num_zero_col=[]
    for i in range(cur_mat.shape[1]):
        col_zero = np.where(cur_mat[:,i].astype(int)==0)[0]
        num_zero_col.append(len(col_zero))

    return num_zero_col,num_zero_row

#print(cur_mat)
# optimality test


def optimality_test(tmp_mat,line):
    row_delete=[]
    row_remain=list(np.arange(0,dim))
    col_remain=list(np.arange(0,dim))
    col_delete=[]
    num_zero_col,num_zero_row = count_zero(tmp_mat)
    result = False
    while(1):
        #print(num_zero_col,num_zero_row)
        if result == False:
            if np.max(num_zero_row) >= np.max(num_zero_col):
                #print("continue")
                #print("step ",line)
                tmp_row_delete=np.where(np.array(num_zero_row) == np.max(num_zero_row))[0][0]
                #print("delete row ", num_zero_row,tmp_row_delete)
                tmp_mat[tmp_row_delete]+=4
                #print('matrix',tmp_mat)
                row_delete.append(tmp_row_delete)
                row_remain.remove(tmp_row_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
                print(result == False)
            elif np.max(num_zero_col) > np.max(num_zero_row):
                #print("step ",line)
                tmp_col_delete=np.where(np.array(num_zero_col) == np.max(num_zero_col))[0][0]
                #print("delete col",tmp_col_delete)
                tmp_mat[:,tmp_col_delete]+=4
                #print('matrix',tmp_mat)
                col_delete.append(tmp_col_delete)
                col_remain.remove(tmp_col_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
                #print(result == False)
            line+=1
        else:
            #print("break")
            break
    return tmp_mat,row_delete,col_delete,row_remain,col_remain,line




tmp_mat=cur_mat.copy()
tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)
while(line < dim):
    tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)
    print("run_optimality_test")
    min_value=np.min(tmp_mat)
    #print(min_value,row_delete,col_delete,row_remain,col_remain)
    for i in row_delete:
        for l in col_delete:
            cur_mat[i,l]+=min_value
    for x in row_remain:
        for y in col_remain:
            cur_mat[x,y]-=min_value
    tmp_mat=cur_mat.copy()

print(cur_mat)

True
True
True
True
[[0 1 0 2 3 0 0 0]
 [0 1 0 2 3 0 0 0]
 [1 3 1 0 0 0 0 0]
 [1 3 1 0 0 0 0 0]
 [2 0 3 1 2 0 0 0]
 [2 0 3 1 2 0 0 0]
 [3 2 2 3 1 0 0 0]
 [3 2 2 3 1 0 0 0]]


In [6]:

cur_mat=cur_mat[:,:-dum]
i=1
final_row_index=[]
final_col_index=[]
while(np.min(cur_mat)==0):
    num_zero_col,num_zero_row=count_zero(cur_mat)
    print("num_zero_col & row",num_zero_col,num_zero_row)
    if 1 in num_zero_row:
        #print("step remove row1",i)
        tmp_row=np.where(np.array(num_zero_row) == 1)[0][0]
        tmp_col=np.where(cur_mat[tmp_row].astype(int)==0)[0][0]
        cur_mat[tmp_row] += 1
        cur_mat[:,tmp_col] += 1
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col)
        #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
    elif 1 in num_zero_col:
        print("step remove col1",i)
        tmp_col=np.where(np.array(num_zero_col) == 1)[0][0]
        tmp_row=np.where(cur_mat[:,tmp_col].astype(int)==0)[0][0]
        cur_mat[tmp_row] += 1
        cur_mat[:,tmp_col] += 1
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col) 
        #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
    else:
        min_col=np.min(list(filter(lambda x: x != 0, num_zero_col)))
        min_row=np.min(list(filter(lambda x: x != 0, num_zero_row)))
        if min_row <= min_col:
            #print("step remove row2",i)
            tmp_row=np.where(np.array(num_zero_row) == min_row)[0][0]
            tmp_col=np.where(cur_mat[tmp_row].astype(int)==0)[0][0]
            cur_mat[tmp_row] += 1
            cur_mat[:,tmp_col] += 1
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
            #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)  
        else:
            #print("step remove col2",i)
            tmp_col=np.where(np.array(num_zero_col) == min_col)[0][0]
            tmp_row=np.where(cur_mat[:,tmp_col].astype(int)==0)[0][0]
            cur_mat[tmp_row] += 1
            cur_mat[:,tmp_col] += 1
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
            #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
            
    i+=1

        
            
print(cur_mat, final_row_index,final_col_index)

start=0
#print(final_row_index,final_col_index)

for i in range(number_of_doctor):
    end= start+capacity
    l = list(range(start, end))
    start=end
    patient_index = np.array(np.where(np.isin(final_row_index,l))[0])
    patient_num=np.array(final_col_index)[np.array(patient_index.astype(int))]
    print("doctor"+ str(i+1)+" has patient "+ str(patient_num+1))

    


num_zero_col & row [2, 2, 2, 2, 2] [2, 2, 2, 2, 1, 1, 0, 0]
num_zero_col & row [2, 0, 2, 2, 2] [2, 2, 2, 2, 0, 0, 0, 0]
num_zero_col & row [0, 0, 1, 2, 2] [0, 1, 2, 2, 0, 0, 0, 0]
num_zero_col & row [0, 0, 0, 2, 2] [0, 0, 2, 2, 0, 0, 0, 0]
num_zero_col & row [0, 0, 0, 0, 1] [0, 0, 0, 1, 0, 0, 0, 0]
[[2 3 2 4 5]
 [2 3 2 4 5]
 [3 5 3 2 2]
 [3 5 3 2 2]
 [4 2 5 3 4]
 [3 1 4 2 3]
 [4 3 3 4 2]
 [4 3 3 4 2]] [4, 0, 1, 2, 3] [1, 0, 2, 3, 4]
doctor1 has patient [1 3]
doctor2 has patient [4 5]
doctor3 has patient [2]
doctor4 has patient []


In [ ]:
#https://www.youtube.com/watch?v=Hbnpn_YM3YA